# Credit worthiness model

Import libraries

In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from patsy import dmatrices
from sklearn import metrics

Load pickle and add intercept

In [2]:
df = pd.read_pickle('Z:\PMO\Stephen Bowie\Projects\Underwriting\Model\Cleaned Model Data')

Create dummy columns and intercept

In [3]:
y, X = dmatrices('repo_in_12 ~ brand + poi + mo_pmt + amt_fin + call + auto_hist + emp_length + annual_inc + ltv + down + def_down + acv + down + vdc + trade_roll + pack + gross_profit + purch_type + credit_score',
                 df, return_type='dataframe')
X.columns

Index([u'Intercept', u'brand[T.UP]', u'poi[T.y]', u'call[T.c2000]',
       u'call[T.c1500]', u'call[T.green]', u'call[T.blue]', u'call[T.silver]',
       u'call[T.high]', u'auto_hist[T.pay on time]', u'auto_hist[T.late pay]',
       u'auto_hist[T.repo]', u'purch_type[T.simulcast]', u'purch_type[T.repo]',
       u'purch_type[T.trade]', u'mo_pmt', u'amt_fin', u'emp_length',
       u'annual_inc', u'ltv', u'down', u'def_down', u'acv', u'vdc',
       u'trade_roll', u'pack', u'gross_profit', u'credit_score'],
      dtype='object')

In [4]:
X = X.rename(columns = {'auto_hist[T.pay on time]': 'auto_pay',
                        'auto_hist[T.late pay]': 'auto_late',
                        'auto_hist[T.repo]': 'auto_repo',
                       'purch_type[T.simulcast]': 'pt_simul',
                       'purch_type[T.repo]': 'pt_repo',
                       'purch_type[T.trade]': 'pt_trade',
                       'call[T.c2000]': 'call_c2000',
                       'call[T.c1500]': 'call_c1500',
                       'call[T.green]': 'call_green',
                       'call[T.blue]': 'call_blue',
                       'call[T.silver]': 'call_silver',
                       'call[T.high]': 'call_high',
                       'brand[T.UP]': 'brand_up',
                       'poi[T.y]': 'has_poi',})

Set variables for logistic regression

In [5]:
X.describe()

Intercept     brand_up      has_poi   call_c2000   call_c1500  \
count     4262.0  4262.000000  4262.000000  4262.000000  4262.000000   
mean         1.0     0.149460     0.448146     0.176678     0.201549   
std          0.0     0.356583     0.497362     0.381440     0.401204   
min          1.0     0.000000     0.000000     0.000000     0.000000   
25%          1.0     0.000000     0.000000     0.000000     0.000000   
50%          1.0     0.000000     0.000000     0.000000     0.000000   
75%          1.0     0.000000     1.000000     0.000000     0.000000   
max          1.0     1.000000     1.000000     1.000000     1.000000   

        call_green    call_blue  call_silver    call_high     auto_pay  \
count  4262.000000  4262.000000   4262.00000  4262.000000  4262.000000   
mean      0.129517     0.133975      0.15298     0.130221     0.197794   
std       0.335810     0.340665      0.36001     0.336585     0.398383   
min       0.000000     0.000000      0.00000     0.000000     0.000000   
25%       0.000000     0.000000      0.00000     0.000000     0.000000   
50%       0.000000     0.000000      0.00000     0.000000     0.000000   
75%       0.000000     0.000000      0.00000     0.000000     0.000000   
max       1.000000     1.000000      1.00000     1.000000     1.000000   

           ...          annual_inc          ltv          down      def_down  \
count      ...         4262.000000  4262.000000   4262.000000   4262.000000   
mean       ...        27712.890906     1.534538    897.890819    303.936469   
std        ...        14860.760635     0.219125    910.311450    467.485361   
min        ...            0.000000     0.135147      0.000000      0.000000   
25%        ...        19500.000000     1.406731    400.000000      0.000000   
50%        ...        24960.000000     1.551626    500.000000    200.000000   
75%        ...        33280.000000     1.662876   1000.000000    500.000000   
max        ...       221000.000000     2.728847  12900.000000  13182.140000   

                acv          vdc    trade_roll         pack  gross_profit  \
count   4262.000000  4262.000000   4262.000000  4262.000000   4262.000000   
mean   10702.545753  1559.208588    765.847717  3723.967621   2780.822841   
std     3117.865491   970.670188   1793.468033   802.500845   1325.258959   
min     2195.000000     0.000000 -11500.000000   590.000000  -6326.570000   
25%     8895.000000   999.000000      0.000000  2950.000000   1869.270000   
50%    10495.000000  2299.000000      0.000000  4200.000000   3168.430000   
75%    12495.000000  2299.000000      0.000000  4200.000000   3668.430000   
max    25595.000000  2299.000000  11168.470000  4200.000000   8469.270000   

       credit_score  
count   4262.000000  
mean     570.018067  
std       74.711655  
min       59.000000  
25%      518.000000  
50%      565.000000  
75%      618.000000  
max      831.000000  

[8 rows x 28 columns]

In [6]:
y = np.ravel(y)

Need to incorporate:
- Explicitly handle missing values (in credit score): two separate models for credit score, ignore records with missing purchase type
- Explicitly address/handle outliers (id outliers, model with, model without, compare performance)
- Feature scaling (sklearn preprocessing module)
- Feature evaluation/significance, colinearity test, and selection
- Conditional features (e.g. proof of income should emphasize or add weight to annual income)
- train/test split data
- Pipeline?

Logistic regression model

In [7]:
model = LogisticRegression()
model = model.fit(X,y)

In [8]:
#error rate
error_rate = 1 - model.score(X,y)
null_error_rate = y.mean()
print 'error rate: ', error_rate
print 'null error rate: ', null_error_rate
print 'improvement: ', null_error_rate - error_rate

error rate:  0.273815110277
null error rate:  0.281792585641
improvement:  0.00797747536368


In [14]:
#examine the coefficients
pd.DataFrame(zip(X.columns, np.transpose(model.coef_))).sort_values(1,ascending=False)

0                     1
15        mo_pmt     [0.0064762037201]
13       pt_repo      [0.003211462593]
4     call_c1500    [0.00245001869898]
3     call_c2000    [0.00233875281347]
11     auto_repo    [0.00157664849252]
19           ltv   [0.000916010883757]
0      Intercept   [0.000602992610607]
1       brand_up    [0.00049006030394]
26  gross_profit   [0.000332465155133]
23           vdc   [0.000306191106334]
21      def_down   [0.000279277575278]
24    trade_roll   [0.000191155091792]
14      pt_trade   [0.000104148059721]
25          pack   [3.71886728609e-05]
22           acv  [-2.05574474881e-06]
18    annual_inc  [-3.95712149392e-06]
16       amt_fin   [-0.00018889533307]
20          down  [-0.000352683636536]
12      pt_simul  [-0.000394244128412]
10     auto_late  [-0.000489176261353]
5     call_green  [-0.000631383836043]
2        has_poi  [-0.000799833854942]
6      call_blue   [-0.00117970265717]
9       auto_pay   [-0.00128209407351]
8      call_high    [-0.0016487461856]
7    call_silver   [-0.00229424540438]
27  credit_score   [-0.00380695074751]
17    emp_length    [-0.0240491420086]

In [10]:
# predict class labels for the test set
predicted = model.predict(X)
print predicted

[ 0.  0.  0. ...,  0.  0.  0.]


In [11]:
# generate class probabilities
probs = model.predict_proba(X)
print probs

[[ 0.83552313  0.16447687]
 [ 0.66665161  0.33334839]
 [ 0.69551661  0.30448339]
 ..., 
 [ 0.78154341  0.21845659]
 [ 0.85727757  0.14272243]
 [ 0.88041441  0.11958559]]


In [12]:
print metrics.confusion_matrix(y, predicted)
print metrics.classification_report(y, predicted)

[[2979   82]
 [1085  116]]
             precision    recall  f1-score   support

        0.0       0.73      0.97      0.84      3061
        1.0       0.59      0.10      0.17      1201

avg / total       0.69      0.73      0.65      4262

